In [1]:
M = [(0.326, 0.002), (0.300, 0.002), (0.347, 0.002)]
T = [(93.0, 0.5), (93.0, 0.5), (93.0, 0.5)]
P = [(592, 1), (592, 1), (592, 1)]
V = [(156, 1), (156, 1), (156, 1)]

In [2]:
def t2a(ptorr):
    "Convert pressure in torr to pressure in atm"

    return ptorr/760.


def c2k(tempC):
    "Convert Celsius temperature to Kelvin"

    return tempC + 273.15


def ml2l(vol):
    "Convert mL to L"
    
    return vol/1000.

In [3]:
R = 0.08205746  # l*atm/mol K

In [4]:
mc = M  # nothing to do for m conversion
tc = [(c2k(t), e) for t, e in T]
pc = [(t2a(p), t2a(e)) for p, e in P]
vc = [(ml2l(v), ml2l(e)) for v, e in V]

In [5]:
# calculate partials**2  * delta**2
err_terms = []
results = []
for m, t, p, v in zip(mc, tc, pc, vc):
    #print(m, t, p, v)
    dM2 = ((R*t[0]/(p[0]*v[0]))**2) * m[1]**2
    dT2 = ((R*m[0]/(p[0]*v[0]))**2) * t[1]**2
    dP2 = ((-R*m[0]*t[0]/(v[0]*p[0]**2))**2) * p[1]**2
    dV2 = ((-R*m[0]*t[0]/(p[0]*v[0]**2))**2) * v[1]**2
    err_terms.append([dM2, dT2, dP2, dV2])
    results.append(R*m[0]*t[0]/(p[0]*v[0]))

from math import sqrt
# compute each error term
run = 0
for m, t, p, v in err_terms:
    run +=1
    tot_err = sqrt(sum([m, t, p, v]))
    print("Run %s: dm = %.2f, dt = %.2f, dp = %.2f, dv = %.2f, total err = %.3f" %(run, m, t, p, v, tot_err))

Run 1: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.27, total err = 0.736
Run 2: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.23, total err = 0.705
Run 3: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.30, total err = 0.763


In [6]:
# MW = mRT/PV
results

[80.6050024410629, 74.17638261447506, 85.79734922407613]

# Using named tuples

In [7]:
from collections import namedtuple

In [9]:
mt = namedtuple('mass', ['value', 'error'])
tt = namedtuple('temp', ['value', 'error'])
pt = namedtuple('pressure', ['value', 'error'])
vt = namedtuple('volume', ['value', 'error'])

In [10]:
mc = [mt(val, err) for val, err in M]
tc = [tt(c2k(val), err) for val, err in T]
pc = [pt(t2a(val), t2a(err)) for val, err in P]
vc = [vt(ml2l(val), ml2l(err)) for val, err in V]

In [11]:
# calculate partials**2  * delta**2
err_terms = []
results = []
for m, t, p, v in zip(mc, tc, pc, vc):
    #print(m, t, p, v)
    dM2 = ((R*t.value/(p.value*v.value))**2) * m.error**2
    dT2 = ((R*m.value/(p.value*v.value))**2) * t.error**2
    dP2 = ((-R*m.value*t.value/(v.value*p.value**2))**2) * p.error**2
    dV2 = ((-R*m.value*t.value/(p.value*v.value**2))**2) * v.error**2
    err_terms.append([dM2, dT2, dP2, dV2])
    results.append(R*m.value*t.value/(p.value*v.value))

from math import sqrt
# compute each error term
run = 0
for m, t, p, v in err_terms:
    run +=1
    tot_err = sqrt(sum([m, t, p, v]))
    print("Run %s: dm = %.2f, dt = %.2f, dp = %.2f, dv = %.2f, total err = %.3f" %(run, m, t, p, v, tot_err))

Run 1: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.27, total err = 0.736
Run 2: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.23, total err = 0.705
Run 3: dm = 0.24, dt = 0.01, dp = 0.02, dv = 0.30, total err = 0.763


In [13]:
results

[80.6050024410629, 74.17638261447506, 85.79734922407613]